# Federated Learning

In [1]:
%matplotlib inline

import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from knlms import *
from krls import *
from krls_rff import *
from klms import *
from klms_rff import *
from kernel import Kernel

# Multiprocessing
import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())

Number of processors:  8


In [2]:
def generate_data(num_data):
    # AWGN
    v = 0.1*np.random.normal(0,1,num_data+2) 

    # Initial conditions
    d_true =[0.1 , 0.1]
    d = [d_true[0] + v[0], d_true[1] + v[1]]

    # Grab new data
    new_d_true = lambda d: d.append((0.8 - 0.5 * np.exp(-(d[-1]**2)))*d[-1] - (0.3 + 0.9*np.exp(-(d[-1]**2)))*d[-2] + 0.1*np.sin(np.pi*d[-1]))
    for i in range(2,num_data+2):
        new_d_true(d_true)
        d.append(d_true[-1] + v[i])

    u = np.hstack((np.array(d[0:num_data]).reshape(num_data,1),np.array(d[1:num_data+1]).reshape(num_data,1)))
    d_true = d_true[2::]
    d = d[2::]
    return np.array(u), np.array(d),np.array(d_true)

In [3]:
# Parameters
num_data = 3000
kernel = Kernel(3.73)
K = 10 # K edge clients
step_size = 0.09
reg_coeff = 0.03
threshold = 0.5

In [4]:
# Data prepocessing for K clients
edges_X = []
edges_y = []
for k in range(K):
    u,d,d_true = generate_data(num_data)
    
    edges_X.append(np.array(u))
    edges_y.append(np.array(d_true))

In [6]:
# Federated learning: Synchronous updating rule

l = 50 # l updates for synchronous
c =  np.ones(K)/K # edge weighting, for now, lets try that all data is being received synchronously
D = 25 # Dictionary size
A = 0 # 1 if asynchronous

y = np.zeros(K)
alpha = np.zeros((D,1))
alphas = []

l_idx = 0

for i in tqdm(range(0,num_data)):
    y_hat = np.zeros(K)
    # Local updates

    for k in range(K):
        u_k = edges_X[k][i:i+1]
        d_true_k = edges_y[k][i:i+1]
        if l_idx == 0:
            err_KLMS_RFF,alpha_k,delta = KLMS_RFF(u_k,d_true_k,kernel,step_size,D,alpha_0=alpha)
        else:
            err_KLMS_RFF,alpha_k,delta = KLMS_RFF(u_k,d_true_k,kernel,step_size,D,alpha_0=alphas[k])        
        alphas[k] = alpha_k 
#         y_hat[k] = alpha.T @ delta
        l_idx+=1
        if l_idx == l:
            break
    
    if l_idx == l:
        # Global update
#         y = A * y + (c @ y_hat.T)
        alpha = A * alpha + (c @ alphas.T)

  0%|                                                  | 0/3000 [00:00<?, ?it/s]


ValueError: setting an array element with a sequence.

In [38]:
# Federated learning: Asynchronous updating rule

In [46]:
for i in [0,1,2,3]:
    print('i',i)
    for j in [4,5,6,7]:
        print('j',j)
        if j==5:
            print('hit')
            break

i 0
j 4
j 5
hit
i 1
j 4
j 5
hit
i 2
j 4
j 5
hit
i 3
j 4
j 5
hit
